In [1]:
library(tidyverse)
library(RPostgreSQL)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.3.4     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: DBI


In [2]:
source("utils.R")

In [3]:
sql_dir <- file.path("..", "sql")
list.files(sql_dir)

[1] "basics.sql"              "cohort.sql"             
[3] "icd9.sql"                "lab_tests.sql"          
[5] "lab_unpivot.sql"         "merge_data.sql"         
[7] "population.sql"          "vital_signs.sql"        
[9] "vital_signs_unpivot.sql"

In [4]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

In [5]:
sql <- readr::read_file(file.path(sql_dir, "basics.sql")) %>%
    make_view_sql("basics", "materialized view")

In [6]:
dbSendQuery(con, sql)

<PostgreSQLResult>

In [7]:
sql <- "
select * from basics limit 10;
"
dbGetQuery(con, sql)

hadm_id,icustay_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,gender,weight,saps,sofa,elix_score,vent,vaso,weekday,mort_28_day,icu_los_day
124271,205951,250,MICU,2188-11-12 09:25:47,2188-11-23 01:24:45,1,23.87840,1,2188-11-12,⋯,F,51.2,26,8,10,1,1,3,1,10.665949
190159,221239,252,MICU,2133-03-31 04:24:30,2133-04-10 20:52:09,1,55.06810,1,2133-04-01,⋯,M,87.0,21,12,14,1,1,2,0,10.685868
110404,280836,268,MICU,2198-02-14 23:27:38,2198-02-18 05:26:11,1,65.98633,1,NA,⋯,F,90.3,28,10,15,1,0,3,1,3.248993
106296,206613,269,MICU,2170-11-05 11:05:29,2170-11-08 17:46:57,1,40.10071,1,2170-11-08,⋯,M,69.0,15,4,4,0,1,1,0,3.278796
177445,244460,663,MICU,2143-12-27 20:59:00,2143-12-29 20:04:00,1,57.62720,1,NA,⋯,F,NA,22,10,0,1,1,5,0,1.961806
166245,231578,668,MICU,2183-07-01 08:05:45,2183-07-07 17:00:02,1,86.86662,1,2183-07-02,⋯,F,60.0,35,10,3,1,1,2,1,6.371030
113494,286061,705,MICU,2156-08-23 00:44:29,2156-08-26 14:22:45,1,68.83390,1,NA,⋯,M,76.4,28,6,12,1,1,1,1,3.568241
140286,282832,707,MICU,2115-02-28 21:20:46,2115-03-05 17:40:11,1,88.90240,1,2115-03-01,⋯,F,75.0,27,8,13,1,1,4,0,4.846817
180826,279467,719,MICU,2171-06-18 03:30:32,2171-06-27 22:02:57,1,83.13980,1,NA,⋯,M,70.0,22,4,0,0,0,2,1,9.772512
171046,261862,720,MICU,2160-02-15 21:45:04,2160-04-04 19:57:31,1,51.47794,1,2160-02-25,⋯,F,65.0,17,9,28,1,1,5,0,48.925312


In [8]:
dbDisconnect(con)
dbUnloadDriver(drv)

[1] TRUE

[1] TRUE